In [1]:
import pandas as pd

In [2]:
df_opp = pd.read_csv('opps.csv')
df_event = pd.read_csv('events.csv')

In [3]:
print(df_opp.shape)
print(df_event.shape)

(1000, 7)
(1048575, 7)


In [4]:
df_opp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ACCOUNT_ID  1000 non-null   object
 1   OPP_ID      1000 non-null   object
 2   ORDER_DATE  1000 non-null   object
 3   PRODUCT     1000 non-null   object
 4   QUANTITY    1000 non-null   int64 
 5   START       998 non-null    object
 6   END         998 non-null    object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


In [5]:
df_event.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   ACCOUNT_ID  1048575 non-null  object
 1   COMPANY_ID  1048575 non-null  object
 2   USER_ID     1048575 non-null  object
 3   PROJECT_ID  977461 non-null   object
 4   DATE        1048575 non-null  object
 5   TYPE        1048575 non-null  object
 6   COUNT       1048575 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 56.0+ MB


In [6]:
df_opp.head()

,ACCOUNT_ID,OPP_ID,ORDER_DATE,PRODUCT,QUANTITY,START,END
0,account1,opp1,4/23/20,cd5ba48bb6ce3541492df6f2282ca555a65397c168dc59...,4,4/24/20,4/23/21
1,account1,opp1,4/23/20,1a5a6aac31b1d9e08401bd147df106c600254b2df05a3f...,2,4/24/20,4/23/21
2,account2,opp2,4/16/20,28746a25d12d36a1c0956436cfd6959f0db252e3020928...,1,4/17/20,4/16/21
3,account2,opp2,4/16/20,1a5a6aac31b1d9e08401bd147df106c600254b2df05a3f...,8,4/17/20,4/16/21
4,account3,opp3,4/9/20,1a5a6aac31b1d9e08401bd147df106c600254b2df05a3f...,3,4/10/20,4/9/21


### Cleaning

In [7]:
# Drop any null values from important variables
df_opp.dropna(subset=['START'], inplace=True)

In [8]:
df_event['PROJECT_ID'].fillna(df_event['COMPANY_ID'], inplace=True)

In [9]:
# Convert dates to datetime type
df_event['DATE'] = pd.to_datetime(df_event['DATE'])
df_opp['ORDER_DATE'] = pd.to_datetime(df_opp['ORDER_DATE'])
df_opp['START'] = pd.to_datetime(df_opp['START'])
df_opp['END'] = pd.to_datetime(df_opp['END'])

In [10]:
df_event.head()

,ACCOUNT_ID,COMPANY_ID,USER_ID,PROJECT_ID,DATE,TYPE,COUNT
0,account420,org1,u1,p1,2019-05-10,099664351c56c479154c4b1e649a727e3ac099cc26747c...,3
1,account399,org2,u2,p2,2019-05-06,78478722fa50547376912d1bc1b21d5f5fb60188015342...,1
2,account399,org2,u3,p3,2019-06-24,9e5fd45ed38136db73e76b46ad11a0200b7a4cbaae9bc1...,2
3,account122,org3,u4,p4,2019-04-30,85c11686c1e1d3072f30b05ff74fd93b92c5d37a1b7ba3...,1
4,account61,org4,u5,p5,2019-08-07,31ea88da80c3371a7e70ac8a9299974290c47e83b46170...,1


In [11]:
df_opp.head()

,ACCOUNT_ID,OPP_ID,ORDER_DATE,PRODUCT,QUANTITY,START,END
0,account1,opp1,2020-04-23,cd5ba48bb6ce3541492df6f2282ca555a65397c168dc59...,4,2020-04-24,2021-04-23
1,account1,opp1,2020-04-23,1a5a6aac31b1d9e08401bd147df106c600254b2df05a3f...,2,2020-04-24,2021-04-23
2,account2,opp2,2020-04-16,28746a25d12d36a1c0956436cfd6959f0db252e3020928...,1,2020-04-17,2021-04-16
3,account2,opp2,2020-04-16,1a5a6aac31b1d9e08401bd147df106c600254b2df05a3f...,8,2020-04-17,2021-04-16
4,account3,opp3,2020-04-09,1a5a6aac31b1d9e08401bd147df106c600254b2df05a3f...,3,2020-04-10,2021-04-09


## Create Aggregated View of Usage Metrics

Since each account has many observations for events, we can summarize these to create a single aggregated record for each account.

**Number of Unique Items**  
`PROJECT_ID`, `ORGANIZATION_ID`, `USER_ID` and `EVENT_TYPE` will all be counted as the total of unique items per account.  These variables represent how many of each of these do they have active.  The concept here is that a more active account will have more usage across all of these categories.  Providing the total will help inform the model.

**Time of Activity**  
For `EVENT_TIME`, there are several ways to build a summary of activity.  For example, is the activity increasing over time, is it decreasing? How active are certain days of the week or even certain projects?  However, for this analysis and building the initial model, simply using the `max` value or the latest date will provide a proxy. 

**Total Events**  
Finally, taking the `sum` of events across each account.  In the case of software utilization, a greater number of events indicates more usage.

In [12]:
df_agg = df_event.groupby(['ACCOUNT_ID'], as_index=False).agg(
    {
        'PROJECT_ID':"nunique", # how many unique projects are they using (more is better)
        'COMPANY_ID':"nunique", # how many different unique orgs (more is better)
        'USER_ID':'nunique', # how many total unique users (more is better)
        'DATE':max, # are the using the software recently (more recent is better)
        'TYPE':"nunique", # how many different features are they using (more is better)
        'COUNT':sum # what is their utilization (larger is better)
    }
)

df_agg.head()

,ACCOUNT_ID,PROJECT_ID,COMPANY_ID,USER_ID,DATE,TYPE,COUNT
0,account1,6,1,6,2019-09-23,21,216
1,account10,116,1,19,2019-10-23,309,87814
2,account100,9,1,5,2019-10-29,188,1582
3,account101,3,1,1,2019-09-18,31,158
4,account102,35,1,3,2019-10-30,214,14744


## Add Calculations for Last 30, 90, 180 Days

In [13]:
# in order to use "last" calculations, you need a date based index
df_ts = df_event.set_index('DATE')

In [14]:
df_14 = df_ts.last('14D').groupby('ACCOUNT_ID')[['COUNT']].sum()
df_14.rename(columns={"COUNT": "COUNT_LAST_14"}, inplace=True)

df_30 = df_ts.last('30D').groupby('ACCOUNT_ID')[['COUNT']].sum()
df_30.rename(columns={"COUNT": "COUNT_LAST_30"}, inplace=True)

df_60 = df_ts.last('60D').groupby('ACCOUNT_ID')[['COUNT']].sum()
df_60.rename(columns={"COUNT": "COUNT_LAST_60"}, inplace=True)

In [15]:
df_agg = pd.merge(df_agg, df_14, on="ACCOUNT_ID", how='left')
df_agg = pd.merge(df_agg, df_30, on="ACCOUNT_ID", how='left')
df_agg = pd.merge(df_agg, df_60, on="ACCOUNT_ID", how='left')

# Finally - fill null values with Zeros for future modeling
df_agg.fillna(0, inplace=True)

In [16]:
df_agg.sample(10)

,ACCOUNT_ID,PROJECT_ID,COMPANY_ID,USER_ID,DATE,TYPE,COUNT,COUNT_LAST_14,COUNT_LAST_30,COUNT_LAST_60
97,account195,4,1,1,2019-10-30,47,1010,227.0,634.0,976
180,account280,1,1,1,2019-10-08,11,39,1.0,22.0,39
194,account293,3,1,1,2019-08-16,11,19,2.0,15.0,19
127,account23,16,1,1,2019-10-02,29,77,1.0,42.0,77
393,account481,46,1,18,2019-10-10,180,10996,1226.0,5573.0,10920
98,account197,7,1,4,2019-10-09,130,2787,190.0,1248.0,2787
385,account474,1,1,1,2019-08-20,1,4,0.0,4.0,4
92,account19,34,1,6,2019-10-21,95,1005,128.0,560.0,1005
188,account288,76,1,9,2019-10-01,190,2503,424.0,1437.0,2500
19,account117,7,1,2,2019-10-21,139,6176,516.0,3797.0,6171


## Merge the Data Sets

Now that there is a single aggregated view, we can merge them to build a single observation for each opportunity.

**Note:** Since the might be multiple opportunities per account, the account activity will be merge / duplicated for each opportunity.  We do not have sufficient data to have opportunity-level usage.

In [17]:
# Merge the datasets on Account ID
df = pd.merge(df_opp, df_agg, on="ACCOUNT_ID")

In [18]:
# Add a column for the number of days transpired since the last known event and the renewal date
df['DAYS_LAST_USED'] = (df['ORDER_DATE'] - df['DATE']).dt.days
df.head()

In [20]:
df.shape

(949, 17)